In [1]:
from ortools.sat.python import cp_model

# Number of matches, stadiums, and days
num_matches = 6
num_stadiums = 6
num_days = 17

# Matrix M indicating match requests
M = [
    [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
    [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
    [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
    [0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.],
    [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.],
    [0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1.]
]

# Matrix indicating matches that cannot be within 3 days of each other
no_adjacent_matches = [
    [2, 3, 4, 5],
    [1, 3, 4, 6],
    [1, 2, 5, 6],
    [1, 2, 5, 6],
    [1, 3, 4, 6],
    [2, 3, 4, 5]
]

# Stadium capacities
capacities = [48000, 83000, 53500, 72000, 94000, 76000]

# Creates the model
model = cp_model.CpModel()

# Define variables
matches = {}
for m in range(num_matches):
    for d in range(num_days):
        for s in range(num_stadiums):
            matches[(m, d, s)] = model.NewBoolVar(f"match_{m}_day_{d}_stadium_{s}")

# Constraint: Each match is scheduled exactly once
for m in range(num_matches):
    model.Add(sum(matches[(m, d, s)] for d in range(num_days) for s in range(num_stadiums)) == 1)

# Constraint: Each stadium hosts at most one match per day
for d in range(num_days):
    for s in range(num_stadiums):
        model.Add(sum(matches[(m, d, s)] for m in range(num_matches)) <= 1)

# Constraint: Matches cannot be within 3 days of each other
for m1 in range(num_matches):
    for m2 in no_adjacent_matches[m1]:
        for d in range(num_days - 3):
            if m2 < num_matches:
                model.Add(
                    matches[(m1, d, s)] + matches[(m2, d + 1, s)] +
                    matches[(m2, d + 2, s)] + matches[(m2, d + 3, s)] <= 3
                )

# Objective: Maximize the sum of (stadium capacity * match number assigned to it * M[s][d])
model.Maximize(
    sum(capacities[s] * (m + 1) * M[s][d] * matches[(m, d, s)]
        for m in range(num_matches)
        for d in range(num_days)
        for s in range(num_stadiums))
)

# Creates the solver and solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print("Solution:")
    scheduled = [[0 for _ in range(num_days)] for _ in range(num_stadiums)]
    for d in range(num_days):
        for m in range(num_matches):
            for s in range(num_stadiums):
                if solver.Value(matches[(m, d, s)]) == 1:
                    scheduled[s][d] = m + 1
    print("Scheduled matrix:")
    for row in scheduled:
        print(row)
    print()
    print("Maximized sum of (stadium capacity * match number assigned to it * M[s][d]) =", solver.ObjectiveValue())
else:
    print("No optimal solution found!")


Solution:
Scheduled matrix:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 2, 3, 0, 0, 0, 0, 4, 0, 0, 5, 0, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Maximized sum of (stadium capacity * match number assigned to it * M[s][d]) = 1974000.0
